In [35]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [36]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [37]:
tf.keras.backend.clear_session()

In [38]:
subject = 'São Paulo - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [39]:
data = pd.read_csv('2003_mo_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
# data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,São Paulo - IDH Longevidade,São Paulo - IDH Renda,São Paulo - Produção de Cimento (t),São Paulo - IDH Educacao,São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,...,Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Consumo de Cimento (t)
0,2003-1,0.835043,0.825444,437.972720,0.794676,0.314974,1.034903e+09,4.730940e+07,23.895559,9.819129e+08,...,8.202881e+10,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,782.111
1,2003-2,0.835425,0.825117,434.363720,0.794957,0.317328,1.036092e+09,4.732994e+07,23.901472,9.822666e+08,...,8.222716e+10,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,789.891
2,2003-3,0.835806,0.824790,433.350868,0.795239,0.319840,1.037280e+09,4.735047e+07,23.907385,9.826203e+08,...,8.234287e+10,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,774.309
3,2003-4,0.836188,0.824463,431.825332,0.795521,0.322104,1.038469e+09,4.737101e+07,23.913298,9.829741e+08,...,8.235526e+10,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,782.071
4,2003-5,0.836570,0.824137,433.287298,0.795802,0.324320,1.039658e+09,4.739155e+07,23.919210,9.833278e+08,...,8.245980e+10,0.648337,-5599.317941,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,841.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2022-9,NaN,0.785531,514.812591,NaN,0.588569,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1028.144
237,2022-10,NaN,0.785634,514.856452,NaN,0.585196,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,996.449
238,2022-11,NaN,0.785738,515.215930,NaN,0.581094,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1024.272
239,2022-12,NaN,0.785842,515.784324,NaN,0.576147,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1024.272


In [40]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,São Paulo - IDH Longevidade,São Paulo - IDH Renda,São Paulo - Produção de Cimento (t),São Paulo - IDH Educacao,São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-3.010633,1.975626,-1.410736,-1.905888,-1.192538,-1.718526,-0.426010,-1.485366,-2.737181,-0.804344,...,-1.023713,-0.767541,2.723741,0.364808,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582
1,-2.936064,1.949013,-1.440931,-1.880961,-1.124056,-1.699080,-0.401643,-1.421647,-2.673768,-0.808200,...,-1.023713,-0.762847,2.350880,0.368956,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356
2,-2.861496,1.922400,-1.449405,-1.856034,-1.050974,-1.679635,-0.377275,-1.357928,-2.610355,-0.812056,...,-1.023713,-0.760108,2.123016,0.331110,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895
3,-2.786927,1.895787,-1.462169,-1.831107,-0.985091,-1.660189,-0.352908,-1.294210,-2.546942,-0.815911,...,-1.023713,-0.759815,2.021477,0.244570,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628
4,-2.712359,1.869173,-1.449937,-1.806179,-0.920633,-1.640744,-0.328540,-1.230491,-2.483529,-0.819767,...,-1.023685,-0.757341,1.887113,0.279013,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.032597,-1.293826,-1.209684,1.419465,1.711139,1.107674,-1.587758,-1.453963,0.436685,1.040670,...,1.711283,1.052051,-2.010387,2.953658,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976
188,-0.094017,-1.291312,-1.192369,1.421533,1.711746,1.098119,-1.580975,-1.459558,0.423023,1.034370,...,1.722665,1.023064,-1.870713,3.351323,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005
189,-0.155437,-1.288797,-1.178463,1.423601,1.717654,1.088565,-1.574192,-1.465154,0.409361,1.028069,...,1.735332,0.994034,-1.806230,3.404908,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489
190,-0.216857,-1.286282,-1.166182,1.425669,1.729082,1.079010,-1.567409,-1.470749,0.395699,1.021769,...,1.748418,0.966056,-1.727496,3.671303,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492


In [41]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      706.731
1      697.361
2      859.304
3      762.222
4      763.920
        ...   
236        NaN
237        NaN
238        NaN
239        NaN
240        NaN
Name: São Paulo - Consumo de Cimento (t), Length: 241, dtype: float64

In [42]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,São Paulo - IDH Longevidade,São Paulo - IDH Renda,São Paulo - Produção de Cimento (t),São Paulo - IDH Educacao,São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-3.010633,1.975626,-1.410736,-1.905888,-1.192538,-1.718526,-0.426010,-1.485366,-2.737181,-0.804344,...,-1.023713,-0.767541,2.723741,0.364808,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582
1,-2.936064,1.949013,-1.440931,-1.880961,-1.124056,-1.699080,-0.401643,-1.421647,-2.673768,-0.808200,...,-1.023713,-0.762847,2.350880,0.368956,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356
2,-2.861496,1.922400,-1.449405,-1.856034,-1.050974,-1.679635,-0.377275,-1.357928,-2.610355,-0.812056,...,-1.023713,-0.760108,2.123016,0.331110,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895
3,-2.786927,1.895787,-1.462169,-1.831107,-0.985091,-1.660189,-0.352908,-1.294210,-2.546942,-0.815911,...,-1.023713,-0.759815,2.021477,0.244570,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628
4,-2.712359,1.869173,-1.449937,-1.806179,-0.920633,-1.640744,-0.328540,-1.230491,-2.483529,-0.819767,...,-1.023685,-0.757341,1.887113,0.279013,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.431225,-1.318515,-0.608419,1.396778,1.023937,1.193677,-1.393041,-1.206846,0.733610,1.413038,...,1.328987,1.557720,-0.214006,1.216560,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659
158,1.391258,-1.316685,-0.653197,1.393445,1.033712,1.195577,-1.411276,-1.217300,0.727991,1.392752,...,1.355196,1.561577,-0.434717,1.155373,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771
159,1.351290,-1.314854,-0.699576,1.390112,1.049101,1.197477,-1.429511,-1.227755,0.722372,1.372465,...,1.381167,1.561138,-0.524091,1.056140,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465
160,1.311322,-1.313024,-0.757794,1.386779,1.059564,1.199378,-1.447746,-1.238210,0.716753,1.352178,...,1.408923,1.559643,-0.614500,1.147619,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668


In [43]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      706.731
1      697.361
2      859.304
3      762.222
4      763.920
        ...   
157    757.915
158    923.634
159    756.663
160    872.715
161    844.988
Name: São Paulo - Consumo de Cimento (t), Length: 162, dtype: float64

In [44]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [45]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 64)

In [46]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,São Paulo - IDH Longevidade,São Paulo - IDH Renda,São Paulo - Produção de Cimento (t),São Paulo - IDH Educacao,São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
126,0.352975,-0.675018,1.357021,0.506668,0.718372,0.835331,-0.070173,-0.126212,1.060884,0.965416,...,0.403484,0.557912,0.888984,-0.230612,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243
127,0.331182,-0.669105,1.320237,0.519610,0.803494,0.853864,-0.120590,-0.162580,1.058281,0.995407,...,0.436846,0.616673,0.954254,-0.156322,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828
128,0.309390,-0.663192,1.259517,0.532552,0.892135,0.872396,-0.171007,-0.198948,1.055678,1.025397,...,0.470723,0.676845,1.045217,-0.120008,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225
129,0.287598,-0.657279,1.196227,0.545494,0.945408,0.890929,-0.221425,-0.235316,1.053074,1.055388,...,0.505118,0.738444,1.176395,-0.143194,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522
130,0.265805,-0.651366,1.135312,0.558436,0.989786,0.909462,-0.271842,-0.271684,1.050471,1.085379,...,0.540037,0.801485,1.303259,-0.028712,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.032597,-1.293826,-1.209684,1.419465,1.711139,1.107674,-1.587758,-1.453963,0.436685,1.040670,...,1.711283,1.052051,-2.010387,2.953658,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976
188,-0.094017,-1.291312,-1.192369,1.421533,1.711746,1.098119,-1.580975,-1.459558,0.423023,1.034370,...,1.722665,1.023064,-1.870713,3.351323,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005
189,-0.155437,-1.288797,-1.178463,1.423601,1.717654,1.088565,-1.574192,-1.465154,0.409361,1.028069,...,1.735332,0.994034,-1.806230,3.404908,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489
190,-0.216857,-1.286282,-1.166182,1.425669,1.729082,1.079010,-1.567409,-1.470749,0.395699,1.021769,...,1.748418,0.966056,-1.727496,3.671303,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492


In [47]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 64)

In [48]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [49]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [50]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [51]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2188936474, 3955334881, 3249340032, 848512984, 3223240238, 268568371, 210062905, 1989673269, 850757168, 2914670942, 4029899407, 1553045642, 3027204882, 3770471620, 989375956, 427291977, 1856824279, 2549992431, 4283207422, 69368131, 2053148392, 1738655769, 2474535089, 2725573253, 1755482129, 2299502355, 3455649907, 4169015294, 956080109, 4114079916, 3365155002, 648028593, 1503901709, 4284685453, 2434644702, 4212460507, 2751419152, 2501647024, 1111118931, 589180033, 1249931017, 1508739194, 745354775, 1075167850, 3556096476, 4240678041, 1054948743, 1805881343, 3907702664, 3622774534]


Step: 0 ___________________________________________
val_loss: 12032.1923828125
winner_seed: 2188936474


Step: 1 ___________________________________________
val_loss: 11961.021484375
winner_seed: 3955334881


Step: 2 ___________________________________________
val_loss: 17493.74609375


Step: 3 ___________________________________________
val_loss: 18598.24609375


Step: 4 __________________________________

In [52]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 132ms/step - loss: 872745.2500 - val_loss: 22828.5195
Epoch 2/10000
4/4 [==============================] - 0s 12ms/step - loss: 26540.1875 - val_loss: 22386.9297
Epoch 3/10000
4/4 [==============================] - 0s 13ms/step - loss: 26194.5547 - val_loss: 18656.0176
Epoch 4/10000
4/4 [==============================] - 0s 13ms/step - loss: 27175.0137 - val_loss: 18601.7422
Epoch 5/10000
4/4 [==============================] - 0s 13ms/step - loss: 26601.7676 - val_loss: 19599.7969
Epoch 6/10000
4/4 [==============================] - 0s 15ms/step - loss: 27355.0566 - val_loss: 35960.7305
Epoch 7/10000
4/4 [==============================] - 0s 14ms/step - loss: 25628.9746 - val_loss: 18890.6992
Epoch 8/10000
4/4 [==============================] - 0s 14ms/step - loss: 28315.4355 - val_loss: 19587.9414
Epoch 9/10000
4/4 [==============================] - 0s 14ms/step - loss: 26335.6953 - val_loss: 20609.5801
Epoch 10/10000
4/4 [======

4/4 [==============================] - 0s 12ms/step - loss: 23081.7070 - val_loss: 23290.0781
Epoch 77/10000
4/4 [==============================] - 0s 12ms/step - loss: 22727.8145 - val_loss: 21666.4414
Epoch 78/10000
4/4 [==============================] - 0s 14ms/step - loss: 26266.0215 - val_loss: 16321.9170
Epoch 79/10000
4/4 [==============================] - 0s 13ms/step - loss: 24298.8379 - val_loss: 17417.7109
Epoch 80/10000
4/4 [==============================] - 0s 12ms/step - loss: 22322.4219 - val_loss: 22029.4941
Epoch 81/10000
4/4 [==============================] - 0s 13ms/step - loss: 22515.1621 - val_loss: 17044.2949
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 24062.3223 - val_loss: 16894.5762
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 22428.0996 - val_loss: 21828.3711
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 21599.4922 - val_loss: 20655.5762
Epoch 85/10000
4/4 [==============

Epoch 151/10000
4/4 [==============================] - 0s 12ms/step - loss: 11730.6396 - val_loss: 17856.1660
Epoch 152/10000
4/4 [==============================] - 0s 14ms/step - loss: 11787.4668 - val_loss: 12062.7305
Epoch 153/10000
4/4 [==============================] - 0s 16ms/step - loss: 11400.0322 - val_loss: 13293.1934
Epoch 154/10000
4/4 [==============================] - 0s 14ms/step - loss: 10494.2129 - val_loss: 16002.5605
Epoch 155/10000
4/4 [==============================] - 0s 14ms/step - loss: 9835.5889 - val_loss: 11492.8359
Epoch 156/10000
4/4 [==============================] - 0s 13ms/step - loss: 11257.3350 - val_loss: 20900.9043
Epoch 157/10000
4/4 [==============================] - 0s 14ms/step - loss: 10361.7998 - val_loss: 9362.4199
Epoch 158/10000
4/4 [==============================] - 0s 13ms/step - loss: 9900.2363 - val_loss: 9929.7422
Epoch 159/10000
4/4 [==============================] - 0s 13ms/step - loss: 13914.9170 - val_loss: 11231.7979
Epoch 160/1000

Epoch 226/10000
4/4 [==============================] - 0s 15ms/step - loss: 10013.7451 - val_loss: 24122.8770
Epoch 227/10000
4/4 [==============================] - 0s 14ms/step - loss: 9045.6152 - val_loss: 16167.8174
Epoch 228/10000
4/4 [==============================] - 0s 12ms/step - loss: 10082.8047 - val_loss: 20567.3281
Epoch 229/10000
4/4 [==============================] - 0s 12ms/step - loss: 8236.2129 - val_loss: 18024.0918
Epoch 230/10000
4/4 [==============================] - 0s 14ms/step - loss: 8549.4395 - val_loss: 14665.6514
Epoch 231/10000
4/4 [==============================] - 0s 13ms/step - loss: 10275.4238 - val_loss: 16349.3857
Epoch 232/10000
4/4 [==============================] - 0s 13ms/step - loss: 8375.8623 - val_loss: 12190.2891
Epoch 233/10000
4/4 [==============================] - 0s 14ms/step - loss: 9109.1787 - val_loss: 14114.1641
Epoch 234/10000
4/4 [==============================] - 0s 13ms/step - loss: 8702.2021 - val_loss: 11628.1064
Epoch 235/10000


Epoch 301/10000
4/4 [==============================] - 0s 13ms/step - loss: 11796.1992 - val_loss: 11339.4922
Epoch 302/10000
4/4 [==============================] - 0s 13ms/step - loss: 9231.0996 - val_loss: 15412.6562
Epoch 303/10000
4/4 [==============================] - 0s 12ms/step - loss: 9351.5117 - val_loss: 30603.7559
Epoch 304/10000
4/4 [==============================] - 0s 13ms/step - loss: 11602.6191 - val_loss: 12633.2529
Epoch 305/10000
4/4 [==============================] - 0s 12ms/step - loss: 13391.6631 - val_loss: 28144.2285
Epoch 306/10000
4/4 [==============================] - 0s 13ms/step - loss: 9466.1543 - val_loss: 12899.4463
Epoch 307/10000
4/4 [==============================] - 0s 12ms/step - loss: 9847.9512 - val_loss: 16061.5791
Epoch 308/10000
4/4 [==============================] - 0s 13ms/step - loss: 9137.7490 - val_loss: 10641.8281
Epoch 309/10000
4/4 [==============================] - 0s 12ms/step - loss: 12361.4258 - val_loss: 12368.1455
Epoch 310/10000

...

4/4 [==============================] - 0s 13ms/step - loss: 8243.6572 - val_loss: 14602.5137
Epoch 1420/10000
4/4 [==============================] - 0s 14ms/step - loss: 8961.3174 - val_loss: 14069.8584
Epoch 1421/10000
4/4 [==============================] - 0s 14ms/step - loss: 9567.0938 - val_loss: 9386.5195
Epoch 1422/10000
4/4 [==============================] - 0s 14ms/step - loss: 9878.7627 - val_loss: 11949.7021
Epoch 1423/10000
4/4 [==============================] - 0s 14ms/step - loss: 10055.4297 - val_loss: 10305.8203
Epoch 1424/10000
4/4 [==============================] - 0s 13ms/step - loss: 9415.9482 - val_loss: 12535.7080
Epoch 1425/10000
4/4 [==============================] - 0s 12ms/step - loss: 10609.1484 - val_loss: 13578.8945
Epoch 1426/10000
4/4 [==============================] - 0s 12ms/step - loss: 8038.2573 - val_loss: 13346.3262
Epoch 1427/10000
4/4 [==============================] - 0s 13ms/step - loss: 8357.3486 - val_loss: 11254.3857
Epoch 1428/10000
4/4 [====

4/4 [==============================] - 0s 14ms/step - loss: 8466.4004 - val_loss: 16281.4043
Epoch 1494/10000
4/4 [==============================] - 0s 13ms/step - loss: 8506.6025 - val_loss: 22302.4590
Epoch 1495/10000
4/4 [==============================] - 0s 13ms/step - loss: 7995.6406 - val_loss: 18692.9668
Epoch 1496/10000
4/4 [==============================] - 0s 14ms/step - loss: 7813.4077 - val_loss: 17723.9277
Epoch 1497/10000
4/4 [==============================] - 0s 15ms/step - loss: 7682.8154 - val_loss: 16208.2012
Epoch 1498/10000
4/4 [==============================] - 0s 15ms/step - loss: 7328.6660 - val_loss: 14213.7451
Epoch 1499/10000
4/4 [==============================] - 0s 14ms/step - loss: 8094.4277 - val_loss: 14614.8545
Epoch 1500/10000
4/4 [==============================] - 0s 13ms/step - loss: 7729.9644 - val_loss: 18858.5566
Epoch 1501/10000
4/4 [==============================] - 0s 13ms/step - loss: 8105.0078 - val_loss: 12743.3721
Epoch 1502/10000
4/4 [=====

4/4 [==============================] - 0s 13ms/step - loss: 9716.4248 - val_loss: 16842.1133
Epoch 1568/10000
4/4 [==============================] - 0s 14ms/step - loss: 7695.1250 - val_loss: 14406.4404
Epoch 1569/10000
4/4 [==============================] - 0s 13ms/step - loss: 9359.4189 - val_loss: 20560.5840
Epoch 1570/10000
4/4 [==============================] - 0s 12ms/step - loss: 9791.3848 - val_loss: 13599.8125
Epoch 1571/10000
4/4 [==============================] - 0s 13ms/step - loss: 8773.8848 - val_loss: 11406.2168
Epoch 1572/10000
4/4 [==============================] - 0s 12ms/step - loss: 7490.0962 - val_loss: 13084.9668
Epoch 1573/10000
4/4 [==============================] - 0s 13ms/step - loss: 8979.1025 - val_loss: 13738.4375
Epoch 1574/10000
4/4 [==============================] - 0s 13ms/step - loss: 8719.8301 - val_loss: 13697.1191
Epoch 1575/10000
4/4 [==============================] - 0s 12ms/step - loss: 8585.3955 - val_loss: 16255.9229
Epoch 1576/10000
4/4 [=====

4/4 [==============================] - 0s 12ms/step - loss: 7167.5464 - val_loss: 11818.9004
Epoch 1642/10000
4/4 [==============================] - 0s 13ms/step - loss: 7941.6489 - val_loss: 20329.4219
Epoch 1643/10000
4/4 [==============================] - 0s 12ms/step - loss: 9765.9580 - val_loss: 15889.2617
Epoch 1644/10000
4/4 [==============================] - 0s 14ms/step - loss: 12718.5918 - val_loss: 21737.9219
Epoch 1645/10000
4/4 [==============================] - 0s 10ms/step - loss: 11337.7383 - val_loss: 12466.1982
Epoch 1646/10000
4/4 [==============================] - 0s 13ms/step - loss: 8934.7764 - val_loss: 10641.6113
Epoch 1647/10000
4/4 [==============================] - 0s 13ms/step - loss: 8989.7656 - val_loss: 18180.7910
Epoch 1648/10000
4/4 [==============================] - 0s 13ms/step - loss: 7593.3911 - val_loss: 24589.3594
Epoch 1649/10000
4/4 [==============================] - 0s 14ms/step - loss: 8564.8574 - val_loss: 17778.3125
Epoch 1650/10000
4/4 [===

4/4 [==============================] - 0s 14ms/step - loss: 9053.9639 - val_loss: 14318.5400
Epoch 1716/10000
4/4 [==============================] - 0s 13ms/step - loss: 7569.1992 - val_loss: 12340.8887
Epoch 1717/10000
4/4 [==============================] - 0s 13ms/step - loss: 7475.6201 - val_loss: 12374.1406
Epoch 1718/10000
4/4 [==============================] - 0s 13ms/step - loss: 7674.1167 - val_loss: 12852.3301
Epoch 1719/10000
4/4 [==============================] - 0s 14ms/step - loss: 7278.4248 - val_loss: 11066.4736
Epoch 1720/10000
4/4 [==============================] - 0s 14ms/step - loss: 7610.4595 - val_loss: 10987.5107
Epoch 1721/10000
4/4 [==============================] - 0s 12ms/step - loss: 7375.8726 - val_loss: 12641.4482
Epoch 1722/10000
4/4 [==============================] - 0s 13ms/step - loss: 7679.4014 - val_loss: 18812.8223
Epoch 1723/10000
4/4 [==============================] - 0s 12ms/step - loss: 7001.5645 - val_loss: 14246.1201
Epoch 1724/10000
4/4 [=====

In [53]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [54]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,826.266602,828.22821,843.503113,916.715332,913.638184,911.342834,894.428223,881.583496,887.294067,886.267334,840.987915,882.634277,882.832458,891.207397,896.42926,887.619141,891.901428,886.4646,876.512146,881.787231,885.460205,902.205505,903.84198,904.115601,904.30365,904.435181,904.555115,904.68811,904.846619,904.999512
Target,910.653,937.696,871.611,795.853,804.767,712.733,763.665,727.397,855.245,855.042,648.867,1003.807,855.104,913.329,849.262,897.883,860.059,771.417,860.232,773.607,790.146,886.929,941.6,817.301,957.289,978.99,906.956,980.948,872.046,692.232
Error,84.386414,109.467773,28.10791,120.862305,108.871155,198.609863,130.763245,154.186523,32.049072,31.225342,192.120911,121.172729,27.728455,22.121582,47.167236,10.263855,31.842407,115.047607,16.280151,108.180237,95.314209,15.276489,37.757996,86.814575,52.985352,74.55481,2.400879,76.259888,32.800598,212.767517


In [55]:
display(mae)
display(mape)

79.24623

0.09970146

In [56]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [57]:
errors, mae, mape = year_mae_mape_calculator(trained_model,
                                             reshaped_test, 
                                             reshaped_test_target,
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 20ms/step
Ano-0: |Prediction[[10512.891]] - Target[9887.336000000001]| =  Error: [[625.5547]]; MAPE:[[0.06326827]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[10690.376]] - Target[10216.868999999999]| =  Error: [[473.50684]]; MAPE:[[0.04634559]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[5427.828]] - Target[5388.461]| =  Error: [[39.367188]]; MAPE:[[0.00730583]]


[array([[625.5547]], dtype=float32),
 array([[473.50684]], dtype=float32),
 array([[39.367188]], dtype=float32)]

379.47623

0.03897323